In [1]:
from itertools import combinations 
from random import shuffle 
import numpy as np 
import sys

In [2]:
# general stuff that will work for any card game 
# cards and deck 
##############################################################

display_suits = [ 's', 'h', 'c', 'd' ]
icon_suits =    [ '♠', '♡', '♣', '♢' ] 
display_ranks = [ 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K' ]
vals =          [  1 , 2, 3, 4, 5, 6, 7, 8, 9, 10,  10,  10,  10 ]
map_suits = dict(zip(range(4),display_suits))
map_ranks = dict(zip(range(13),display_ranks))
map_icons = dict(zip(range(4),icon_suits))
map_vals = dict(zip(range(13),vals))

class Card:
    
    def __repr__( self ):
        return '{}{}'.format( self.display_rank, self.display_suit )
    
    # as card symbols 
    def __repr__(self):
        return '{}{}'.format(self.display_rank, self.icon_suit) 
    
    def __init__( self, suit, rank ):
        self.suit = suit
        self.rank = rank 
        self.index = (13*self.suit)+self.rank
        self.display_rank = map_ranks[ self.rank ]
        self.display_suit = map_suits[ self.suit ]
        self.icon_suit = map_icons[ self.suit ] 
        self.value = map_vals[ self.rank ]
        
class Deck:
    def __init__( self ):
        self.cards = []
        for s in range( 4 ):
            for r in range( 13 ):
                self.cards.append(Card(s, r))
                
    def shuffle( self ):
        shuffle( self.cards )
        
    def draw( self, n=1 ):
        result = []
        for i in range( n or 1 ):
            result.append( self.cards.pop() ) 
        return result 

In [3]:
deck = Deck()
deck.shuffle()
deck.draw()

[2♣]

In [4]:
deck.draw()

[10♣]

In [5]:
# players 
#######################################################

class Player():
    def __init__( self, name='' ): 
        self.name = name 
        self.hand = []
        self.score = 0

    def clean( self ):
        self.hand = []
        self.score = 0 
        
    def peg( self, points ):
        self.score += points 
        # note: works to decrement as well 
        # player.peg(-2) #for muggins or something 
    
    @property 
    def sorted_hand(self):
        return sorted(self.hand, key=lambda c: c.value)
            
class HumanPlayer(Player):
    def ask_for_input( self, hand ):
        d = dict(enumerate(hand,1))
        discard_prompt = 'Your hand: ' + ' '.join([str(x) for x in d.values()]) + '\nChoose a card to play: '
        inp = input( discard_prompt )
        card = d[ int(inp) ]
        return card 
    
    def ask_for_discards( self ):
        d = dict(enumerate(self.sorted_hand,1))
        discard_prompt = 'Your hand: {} {} {} {} {} {}\nChoose two cards (numbered 1-6) for the crib: '.format(*d.values())
        inp = input( discard_prompt )
        cards = [ d[int(i)] for i in inp.replace(' ','') ]
        self.hand = [ n for n in self.hand if n not in cards ] 
        print( 'Discarded {} {} to crib'.format(*cards) )
        return cards

class NondeterministicAIPlayer(Player):
    def ask_for_input(self, hand):
        card = np.random.choice(hand) 
        return card
    def ask_for_discards(self):
        cards = self.hand[0:2]
        self.hand = [ n for n in self.hand if n not in cards ] 
        return cards
    
# class NondeterministicAIPlayer(Player):
class AIPlayer(Player):
    def ask_for_input(self, hand):
        card = np.random.choice(hand) 
        # not a very smart AI
        # can it fool will? 
        return card

In [6]:
######################################################
######## macro parts of the cribbage game

In [7]:
def discards( players ):
    crib = []
    for player in players:            
        discards = player.ask_for_discards()
        for c in discards:
            crib.append(c) 
    return crib 

In [8]:
def counting( players, with_gui=False ):
    
    # counting functions 
    def play( card ):
        nonlocal count, turn, hands
        plays.append( card ) 
        count += card.value
        hands[turn] = [ n for n in hands[turn] if n != card ]         
        #print( card, 'was played by player', turn ) 
        return count, hands 

    def say_go():
        nonlocal score, turn  
        score[turn^1] += 1 
    
    def validate_play( card, count ):
        if card.value + count < 32:
            return True 
        else:
            return False 
    
    # game logic         
    hands = [player.hand for player in players]
    turn = np.random.randint(2) # either 0 or 1 
    count = 0
    plays = []
    play_runs = []
    score = np.array([0,0]) # get a player's score like score[player_number] 

    while all(hands): # while any hand is not empty 
        while True: # we are counting toward 31 
            if with_gui:
                fmt_str = 'Turn: {} Count: {} Score: {} {}' 
                print( fmt_str.format( turn, count, *score ) )

            # first, does whomever's turn it is even have cards? 
            if not hands[turn]:
                break # out of while game_on 
            # first, is whoever's turn it is even have a legal move
            # or do they have to say "go"? 
            if all([count+card.value>31 for card in hands[turn]]):
                # they have to say "go" 
                # other person gets 1 point 
                score[turn^1] += 1 
                if with_gui:
                    print( "Go --- player", turn ) 
                break # out of "while game_on" 

            # generic turn 
            done = False
            while not done: 
                card = players[turn].ask_for_input(hands[turn])
                done = validate_play(card, count)
            play(card) 

            # score and set up for next iteration 
            # current player is 
            if count == 31:
                # we are done, son 
                score[ turn ] += 2 # 31 for 2 
                break 
            if count == 15:
                score[ turn ] += 2 # 15 for 2 

            # swap turn     
            turn ^= 1 # swap turn 

        # anything you want to happen after we stop couting to 31 
        #print( "Plays to 31:", plays ) 
        play_runs.append( plays ) 
        count = 0 
        plays = []
        
    # anything you want to happen after we don't have any cards left 
    # return from "counting" function 
    return score 

In [9]:
# cribbage scoring 
# ai may never know this 
def score( hand ):

    points = 0
    assert len(hand) == 5

    # fifteens  
    for vector_length in [ 2, 3, 4, 5 ]:
        for vector in combinations( hand, vector_length ):
            if sum( [ card.value for card in vector ] ) == 15:
                points += 2 

    # pairs (not necessary to account for more than pairs for ==)
    for i, j in combinations( hand, 2 ):
        if i.rank == j.rank:
            points += 2 

    # runs
    for vector_len in [ 5, 4, 3 ]:
        for vec in combinations( hand, vector_len ):
            vals = [ card.value for card in vec ]
            run = [ n + min( vals ) for n in range( vector_len ) ]
            if sorted( vals ) == run:
                points += vector_len
                break
            break 
                      
    return points 



In [10]:
def discards( players ):
    crib = []
    for player in players:            
        discards = player.ask_for_discards()
        for c in discards:
            crib.append(c) 
    return crib 

def scoring(players, turn_card):
    # goes with "score" function 
    for player in players:
        pool = player.hand + turn_card 
        player.score += score(pool) 

In [11]:
#################################################
# actual game 

In [12]:
# a game without any user input 

def game_without_gui(players):
    while players[0].score < 121 and players[1].score < 121:
        turn = np.random.randint(2)
        deck = Deck()
        deck.shuffle()
        for player in players:
            player.hand = deck.draw(6)    
        crib = discards(players)        # decisions 
        counting(players, with_gui=False)               # decisions 
        turn_card = deck.draw(1)
        scoring(players, turn_card)
        players[turn^1].score += score(crib+turn_card)
    
    return_val = [ player.score for player in players ] 
    for player in players:
        player.clean()
    return return_val 

In [13]:
#p = [NondeterministicAIPlayer('p1'), NondeterministicAIPlayer('p2')]
p = [NondeterministicAIPlayer(), NondeterministicAIPlayer('will')]
[game_without_gui(p) for n in range(10)]

[[121, 121],
 [128, 103],
 [61, 124],
 [112, 123],
 [122, 126],
 [125, 81],
 [82, 123],
 [123, 93],
 [122, 94],
 [127, 97]]

In [ ]:
def game_with_gui(players):
    
    p1, p2 = players #human first, by convention 
    
    while p1.score < 121 and p1.score < 121:
        
        turn = np.random.randint(2) # 0 or 1, whose crib is it? 
        print( 'Dealer is player', turn ) 
        print( 'Shuffling deck and dealing cards' ) 
        deck = Deck()
        deck.shuffle()
        for player in players:
            player.hand = deck.draw(6)    

        # ask players for thier discards 
        crib = discards(players)        # decisions 
        
        # run the counting sub-game 
        counting(players, with_gui=True)               # decisions 
        #assert len( p1.hand ) == len( p2.hand ) == 0 
        print( 'After counting, the score is', p1.score, p2.score )
        
        # turn card and final scoring      
        turn_card = deck.draw(1)
        p1.peg(score(p1.hand+turn_card))
        p2.peg(score(p2.hand+turn_card))
#         scoring(players, turn_card)
        players[turn^1].score += score(crib+turn_card)
        
    
    return_val = [ player.score for player in players ] 
    for player in players:
        player.clean()
    return return_val 

In [ ]:
game_with_gui([HumanPlayer(), NondeterministicAIPlayer()])

Dealer is player 1
Shuffling deck and dealing cards
